In [38]:
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.io.ase import AseAtomsAdaptor

from ase.io import read, write

pristine_slab = read("GaN_0001_3x3_pristine_slab.cif")
pristine_pmg_slab = AseAtomsAdaptor.get_structure(pristine_slab)
site_finder = AdsorbateSiteFinder(pristine_pmg_slab)
# AdsorbateSiteFinder(slab, selective_dynamics=False, height=0.9, mi_vec=None)
sites = site_finder.find_adsorption_sites(put_inside=False, symm_reduce=False)
# find_adsorption_sites(distance=2.0, put_inside=True, symm_reduce=0.01, near_reduce=0.01, positions=['ontop', 'bridge', 'hollow'], no_obtuse_hollow=True)


In [39]:
for type in sites.keys():
    print(f"{type} sites has total {len(sites[type])}")

ontop sites has total 18
bridge sites has total 69
hollow sites has total 8
all sites has total 95


In [40]:
import numpy as np
ads_positions = sites['all']
print(f"adsorption coordinates are")
print(ads_positions)
# take just the first 18 sites after inspection
select_positions = ads_positions[:18]

adsorption coordinates are
[array([ 0.        ,  0.        , 15.26569384]), array([12.86517608,  7.42771287, 14.62006789]), array([ 3.21630608,  5.57080555, 15.26569384]), array([11.25704716,  4.64235188, 14.62006789]), array([ 1.60812892,  2.78536099, 15.26569384]), array([ 9.64887   ,  1.85690733, 14.62006789]), array([ 6.43261216,  0.        , 15.26569384]), array([ 9.64891824,  7.42771287, 14.62006789]), array([ 9.64891824,  5.57080555, 15.26569384]), array([ 8.04078933,  4.64235188, 14.62006789]), array([ 8.04074108,  2.78536099, 15.26569384]), array([ 6.43261216,  1.85690733, 14.62006789]), array([ 3.21625784,  0.        , 15.26569384]), array([ 6.43256392,  7.42771287, 14.62006789]), array([ 6.43256392,  5.57080555, 15.26569384]), array([ 4.824435  ,  4.64235188, 14.62006789]), array([ 4.82438676,  2.78536099, 15.26569384]), array([ 3.21625784,  1.85690733, 14.62006789]), array([32.96702879, 14.39119892, 14.62006789]), array([32.96702879, 13.46274525, 14.94288087]), array([33.77

In [41]:
# adsorb onto the sites
from pymatgen.core.structure import Molecule
ads_mol = Molecule(['Ga'], [[0,0,0]])
for ads_coord in select_positions:
    slab = site_finder.add_adsorbate(ads_mol, ads_coord, repeat=None, translate=False, reorient=False)
    site_finder.slab = slab

In [42]:
len(site_finder.slab)

54

In [43]:
from pymatgen.io.cif import CifWriter
# write out slab
cif_slab = CifWriter(site_finder.slab)
cif_slab.write_file("pymatgen_GaN_0001_ads_Ga_all_adsorbed_slab.cif")

Manually check the file, looks legit

In [44]:
# do a sample run
from mcmc import mcmc_run, slab_energy

from ase.calculators.lammpsrun import LAMMPS
from ase.io import read
from ase.build import make_supercell
from catkit.gen.adsorption import get_adsorption_sites

import catkit
import os
import numpy as np

import sys
sys.path.append("/home/dux/")
from htvs.djangochem.pgmols.utils import surfaces

In [45]:
# Get pristine surface
# GaN 0001 surface
atoms = read('GaN_hexagonal.cif')

# supercell_atoms = atoms*(2,2,2)
# supercell_atoms.write('GaN_hexagonal_2x2.cif')

supercell_atoms = atoms*(3,3,3)
supercell_atoms.write('GaN_hexagonal_3x3.cif')

slab, surface_atoms = surfaces.surface_from_bulk(supercell_atoms, [0,0,0,-1], size=[3,3], vacuum=10)

# try 2003 tersoff potential 
parameters = {
    'pair_style': 'tersoff',
    'pair_coeff': ['* * GaN.tersoff Ga N']
}
potential_file = os.path.join(os.environ["LAMMPS_POTENTIALS"], 'GaN.sw')
lammps_calc = LAMMPS(files=[potential_file], keep_tmp_files=False, keep_alive=False, tmp_dir="/home/dux/surface_sampling/tmp_files")
lammps_calc.set(**parameters)

element = 'Ga'
ads = catkit.gratoms.Gratoms(element)

# starting from more random initial positions
num_ads_atoms = 12 # needs to have so many atoms

slab, surface_atoms = surfaces.surface_from_bulk(supercell_atoms, [0,0,0,-1], size=[3,3], vacuum=10)
# get initial adsorption sites
proper_adsorbed = read("GaN_0001_3x3_12_Ga_ads_initial_slab.cif")
ads_positions = proper_adsorbed.get_positions()[len(slab):]
assert len(ads_positions) == num_ads_atoms, "num of adsorption sites does not match num ads atoms"

# canonical with relaxation
num_runs = 1
surface_name = "GaN_0001_3x3"
alpha = 0.99
slab, surface_atoms = surfaces.surface_from_bulk(supercell_atoms, [0,0,0,-1], size=[3,3], vacuum=10)
# set surface atoms from the other side
all_atoms = np.arange(len(slab))
curr_surf_atoms = slab.get_surface_atoms()
new_surf_atoms = np.setdiff1d(all_atoms, curr_surf_atoms)
slab.set_surface_atoms(new_surf_atoms)
# invert the positions
slab.set_scaled_positions(1 - slab.get_scaled_positions())

# try positive chem pot
chem_pot = 5
history, energy_hist, frac_accept_hist, adsorption_count_hist = mcmc_run(num_runs=num_runs, temp=1, pot=chem_pot, alpha=alpha, slab=slab, calc=lammps_calc, surface_name=surface_name, element=element, canonical=True, num_ads_atoms=num_ads_atoms, relax=True, ads_coords=select_positions)

/home/dux/.local/lib/python3.9/site-packages/catkit/gen/surface.py:334: UserWarning: Your slab has no bulk atoms and may be too thin to identify surface atoms correctly. This may cause surface adsorption site identification to fail.
  warnings.warn(
INFO:Running with num_runs = 1, temp = 1, pot = 5, alpha = 0.99
INFO:there are 36 atoms 
INFO:using slab calc <ase.calculators.lammpsrun.LAMMPS object at 0x7f213ae2d1c0>
/home/dux/.local/lib/python3.9/site-packages/catkit/gen/adsorption.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r1_topology = np.array(self.r1_topology)
/home/dux/.local/lib/python3.9/site-packages/catkit/gen/adsorption.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tupl

      Step     Time          Energy         fmax
BFGS:    0 18:52:51     -144.841522        4.8071
BFGS:    1 18:52:51     -145.120115        3.3709
BFGS:    2 18:52:51     -145.469519        1.1921


/home/dux/miniconda3/envs/mlenv/lib/python3.9/site-packages/ase/calculators/lammpsrun.py:379: UserWarning: semi-periodic ASE cell detected - translation to proper LAMMPS input cell might fail
  warnings.warn(


BFGS:    3 18:52:51     -145.540834        0.4665
BFGS:    4 18:52:51     -145.556943        0.1145
      Step     Time          Energy         fmax
BFGS:    0 18:52:51     -145.529722        4.7798
BFGS:    1 18:52:51     -146.080998        3.3575
BFGS:    2 18:52:51     -146.777665        1.1928
BFGS:    3 18:52:51     -146.921507        0.4737
BFGS:    4 18:52:51     -146.954835        0.1326
      Step     Time          Energy         fmax
BFGS:    0 18:52:51     -146.479085        4.7798
BFGS:    1 18:52:51     -147.088757        3.3575
BFGS:    2 18:52:51     -147.950971        1.1708
BFGS:    3 18:52:51     -148.149563        0.7924
BFGS:    4 18:52:51     -148.204954        0.5291
BFGS:    5 18:52:51     -148.214213        0.4065
BFGS:    6 18:52:51     -148.219337        0.3175
BFGS:    7 18:52:51     -148.227960        0.2020
BFGS:    8 18:52:51     -148.236892        0.1467
      Step     Time          Energy         fmax
BFGS:    0 18:52:51     -145.529722        4.7798
BFG

INFO:running for 18 iterations per run over a total of 1 runs
INFO:adsorbate is Ga
INFO:In sweep 1 out of 1


      Step     Time          Energy         fmax
BFGS:    0 18:52:59     -150.410130        6.4650
BFGS:    1 18:53:00     -154.496772        5.2315
BFGS:    2 18:53:00     -160.656855        3.9117
BFGS:    3 18:53:00     -165.651989        5.8159
BFGS:    4 18:53:00     -168.438650        2.2212
BFGS:    5 18:53:00     -169.274538        1.8028
BFGS:    6 18:53:00     -169.419633        1.1347
BFGS:    7 18:53:00     -169.555824        0.8501
BFGS:    8 18:53:00     -169.711532        0.9895
BFGS:    9 18:53:00     -169.755514        0.6059
BFGS:   10 18:53:00     -169.803032        0.8452
BFGS:   11 18:53:00     -169.835781        0.3891
BFGS:   12 18:53:00     -169.873469        0.3230
BFGS:   13 18:53:00     -169.894265        0.3497
BFGS:   14 18:53:00     -169.917275        0.4004
BFGS:   15 18:53:00     -169.933339        0.3650
BFGS:   16 18:53:00     -169.945066        0.3298
BFGS:   17 18:53:00     -169.959009        0.2634
BFGS:   18 18:53:00     -169.970624        0.2148
B

INFO:optim structure has Energy = -173.4969123735532


In [ ]:
runs = range(1, num_runs+1)

# do the plots
fig, ax = plt.subplots(2, 2, figsize=(10, 8))
ax[0, 0].plot(runs, energy_hist)
ax[0, 0].set_xlabel("Iter")
ax[0, 0].set_ylabel("Energy (E)")
ax[0, 0].set_title("Energy (E) vs Sweeps")

ax[0, 1].plot(runs, frac_accept_hist)
ax[0, 1].set_xlabel("Iter")
ax[0, 1].set_ylabel("Fraction accepted")
ax[0, 1].set_title("Fraction accepted vs Sweeps")

ax[1, 1].plot(runs, np.array(list(adsorption_count_hist.values())).T)
ax[1, 1].set_xlabel("Iter")
ax[1, 1].set_ylabel("Adsorption count")
ax[1, 1].legend(adsorption_count_hist.keys())
ax[1, 1].set_title("Adsorption count vs Iterations")

fig.show()
fig.tight_layout()